In [4]:
from dataclasses import dataclass
@dataclass
class Schema:
    evaluated_at_utc:str="datetime64[ns]"
    evaluated_at_ept:str="datetime64[ns]"
    forecast_hour_beginning_utc:str="datetime64[ns]"
    forecast_hour_beginning_ept:str="datetime64[ns]"
    forecast_area:str="string"
    forecast_load_mw:str="float64"

In [5]:
import pandas as pd
from usagrid import s3

fn = s3.list_files_in_folder("usagrid",folder_name="bronze/PJM/load_forecast")[0]
df = s3.read_pyarrow_from_s3(bucket_name="usagrid",key=fn).to_pandas()

In [6]:
df.head()

,evaluated_at_utc,evaluated_at_ept,forecast_hour_beginning_utc,forecast_hour_beginning_ept,forecast_area,forecast_load_mw
0,12/31/2018 10:45:00 AM,12/31/2018 5:45:00 AM,1/1/2019 5:00:00 AM,1/1/2019 12:00:00 AM,RTO,77016
1,12/31/2018 10:45:00 AM,12/31/2018 5:45:00 AM,1/1/2019 5:00:00 AM,1/1/2019 12:00:00 AM,AEP,13122
2,12/31/2018 10:45:00 AM,12/31/2018 5:45:00 AM,1/1/2019 5:00:00 AM,1/1/2019 12:00:00 AM,DAY,1707
3,12/31/2018 10:45:00 AM,12/31/2018 5:45:00 AM,1/1/2019 5:00:00 AM,1/1/2019 12:00:00 AM,EKPC,1187
4,12/31/2018 10:45:00 AM,12/31/2018 5:45:00 AM,1/1/2019 5:00:00 AM,1/1/2019 12:00:00 AM,ATSI,6724


In [7]:
schema = Schema().__dict__
for k,v in schema.items():
    
    print(k,v)

    df[k] = df[k].astype(v)

evaluated_at_utc datetime64[ns]
evaluated_at_ept datetime64[ns]
forecast_hour_beginning_utc datetime64[ns]
forecast_hour_beginning_ept datetime64[ns]
forecast_area string
forecast_load_mw float64


In [12]:
df_oriented = df.sort_values("forecast_hour_beginning_utc")\
                .reset_index(drop=True)

df_oriented

,evaluated_at_utc,evaluated_at_ept,forecast_hour_beginning_utc,forecast_hour_beginning_ept,forecast_area,forecast_load_mw
0,2018-01-01 22:45:00,2018-01-01 17:45:00,2018-01-01 05:00:00,2018-01-01 00:00:00,EKPC,2699.0
1,2018-01-01 22:45:00,2018-01-01 17:45:00,2018-01-01 05:00:00,2018-01-01 00:00:00,RTO,113534.0
2,2018-01-01 22:45:00,2018-01-01 17:45:00,2018-01-01 05:00:00,2018-01-01 00:00:00,DUQ,1658.0
3,2018-01-01 22:45:00,2018-01-01 17:45:00,2018-01-01 05:00:00,2018-01-01 00:00:00,DAY,2336.0
4,2018-01-01 22:45:00,2018-01-01 17:45:00,2018-01-01 05:00:00,2018-01-01 00:00:00,APS,7582.0
...,...,...,...,...,...,...
5399275,2024-06-19 03:45:00,2024-06-18 23:45:00,2024-06-20 03:00:00,2024-06-19 23:00:00,AEP,17604.0
5399276,2024-06-18 21:45:00,2024-06-18 17:45:00,2024-06-20 03:00:00,2024-06-19 23:00:00,DAY,2548.0
5399277,2024-06-18 21:45:00,2024-06-18 17:45:00,2024-06-20 03:00:00,2024-06-19 23:00:00,ATSI,9865.0
5399278,2024-06-19 03:45:00,2024-06-18 23:45:00,2024-06-20 03:00:00,2024-06-19 23:00:00,ATSI,9865.0


In [14]:
s3.write_pyarrow_to_s3(bucket_name="usagrid",object_key="silver/PJM/PJM_load_forecast.arrow",data=df_oriented)